In [1]:
import polars as pl
import pandas as pd
import os
import numpy as np
import glob

In [2]:
data_dir = './data'
figure_dir = './figures'
dnsscan_data = f'{data_dir}/dnsscan'
mikrotik_measurement = f'{data_dir}/mikrotik-testing'

rate_pub_resolver_multi_pop_files = f'{data_dir}/ratelimits-multiple-pops'

scan_overview_file = f'{dnsscan_data}/scan_overview.csv.gz'
scan_df_file = f'{dnsscan_data}/udp_dataframe_complete_2025-01-06.csv.gz'
scan_df_file_aug = f'{dnsscan_data}/udp_dataframe_complete_2024-08-26.csv.gz'

freq_over_time_df_file = f'{dnsscan_data}/frequency_per_type_over_time.csv'

shapefile = f'{data_dir}/shapefiles/ne_110m_admin_0_countries.shp'

In [3]:
overview_df = pl.read_csv(scan_overview_file,separator=";")
scan_df = pl.read_csv(scan_df_file, separator=";")
scan_df_aug = pl.read_csv(scan_df_file_aug, separator=";")
frequency_odns_over_time = pl.read_csv(freq_over_time_df_file,separator=';')

In [4]:
print("#######################")
print("######  TABLE 1  ######")
print("#######################")
print(scan_df
    .filter(pl.col('response_type') == 'Transparent Forwarder')
    .select(['ip_response', 'org_response'])
    .with_columns([
        pl.when(pl.col('org_response').str.contains('GOOGLE'))
          .then(pl.lit('Google'))
        .when(pl.col('org_response').str.contains('CLOUDFLARENET'))
          .then(pl.lit('Cloudflare'))
        .when(pl.col('org_response').str.contains('OPENDNS'))
          .then(pl.lit('OpenDNS'))
        .when(pl.col('org_response').str.contains('RU-JSCIOT|ID-NIC'))
          .then(pl.lit('Hosting'))
        .otherwise(pl.lit('ISP'))
        .alias('Provider')
    ])
    .group_by(['ip_response', 'Provider'])
    .agg(pl.len().alias('Tfwd. [#]'))
    .with_columns(
        (pl.col('Tfwd. [#]') * 100 / pl.col('Tfwd. [#]').sum()).round(2).alias('Tfwd. [%]'))
    .sort('Tfwd. [#]', descending=True)
    .limit(10)
    .to_pandas()
    .to_latex(float_format="%.2f",index=False))

#######################
######  TABLE 1  ######
#######################
\begin{tabular}{llrr}
\toprule
ip_response & Provider & Tfwd. [#] & Tfwd. [%] \\
\midrule
8.8.8.8 & Google & 341447 & 64.25 \\
1.1.1.1 & Cloudflare & 48313 & 9.09 \\
208.67.222.222 & OpenDNS & 14464 & 2.72 \\
8.8.4.4 & Google & 14115 & 2.66 \\
223.29.207.110 & ISP & 11789 & 2.22 \\
83.220.169.155 & Hosting & 2047 & 0.39 \\
178.233.140.109 & ISP & 1790 & 0.34 \\
203.147.91.2 & ISP & 1634 & 0.31 \\
1.0.0.1 & Cloudflare & 1196 & 0.23 \\
103.88.88.88 & Hosting & 1007 & 0.19 \\
\bottomrule
\end{tabular}



In [5]:
fingerprinting_df = pl.read_csv(f"{data_dir}/fingerprinting/fingerprinting_results.csv", separator=";")

In [6]:
fingerprinting_df = fingerprinting_df.filter(pl.col("router vendor").is_not_null())

In [7]:
fingerprinting_df = fingerprinting_df.with_columns(
    pl.when((pl.col('model version').str.starts_with('CCR')) & (pl.col('router vendor')=='Mikrotik')).then(pl.lit('Core'))
    .when(pl.col('router vendor').is_in(['Hikvision','UNV'])).then(pl.lit('NVR')).otherwise(pl.lit("CPE")).alias('Type')
)

In [8]:
fingerprinting_df = fingerprinting_df.group_by(['router vendor','Type']).agg(pl.len().alias('Devices [#]')).sort(by='Devices [#]',descending=True)

In [9]:
corder={'Mikrotik':1,
 'TP-Link':2,
 'Ubiquiti':3,
 'Fortinet':4,
 'ZTE':5,
 'Cisco':6,
 'Zyxel':7,
 'Huawei':8,
 'D-Link':9,
 'Other':10,
 'Hikvision':11,
 'UNV':12}
fingerprinting_df = fingerprinting_df.with_columns(
    pl.when(pl.col('Devices [#]')>=24).then(pl.col('router vendor')).otherwise(pl.lit('Other'))
).group_by(['router vendor','Type']).agg(pl.col('Devices [#]').sum()).sort('Devices [#]',descending=True).to_pandas().sort_values(by='router vendor',key = lambda vendor: vendor.map(corder))
fingerprinting_df = fingerprinting_df.rename(columns={'router vendor':'Vendor'})

In [10]:
print("#######################")
print("######  TABLE 2  ######")
print("#######################")
print(fingerprinting_df.to_latex(index=False))

#######################
######  TABLE 2  ######
#######################
\begin{tabular}{llr}
\toprule
Vendor & Type & Devices [#] \\
\midrule
Mikrotik & Core & 5569 \\
Mikrotik & CPE & 4362 \\
TP-Link & CPE & 728 \\
Ubiquiti & CPE & 663 \\
Fortinet & CPE & 252 \\
ZTE & CPE & 200 \\
Cisco & CPE & 104 \\
Zyxel & CPE & 102 \\
Huawei & CPE & 58 \\
D-Link & CPE & 24 \\
Other & CPE & 114 \\
Hikvision & NVR & 871 \\
UNV & NVR & 25 \\
\bottomrule
\end{tabular}



In [11]:
# all known response addresses are already directly accessible
response_addresses = (
    scan_df
    .filter(pl.col("response_type") != "Transparent Forwarder")
    .select("ip_response")
    .unique()
)
# we might know some resolver addresses through the a-record as well, but they are part of the shielded resolvers since these resolvers are not publically accessible

# mapping of transparent forwarders to shielded resolvers
tfwd_shielded_df = (
    scan_df
    .filter(pl.col("response_type") == "Transparent Forwarder")
    .filter(~pl.col("ip_response").is_in(response_addresses))
)
# ip addresses of only the shielded resolvers themselves
shielded_resolvers = tfwd_shielded_df.select("ip_response").unique()
top5ases = tfwd_shielded_df.group_by("asn_request").agg(pl.len()).sort(by="len",descending=True).head(5).select(pl.col('asn_request'))
print("#######################")
print("######  TABLE 4  ######")
print("#######################")
print(tfwd_shielded_df.group_by("asn_request").agg(pl.len()).sort(by="len",descending=True)\
      .join(tfwd_shielded_df.filter(pl.col('asn_response').is_in(top5ases)).group_by("asn_response","ip_response").agg(pl.len()).group_by("asn_response").agg(pl.len()).sort(by="len",descending=True),left_on='asn_request',right_on='asn_response')\
      .to_pandas().rename(columns={'asn_request':'ASN','len':'\# of Transp. Fwds.','len_right':'\# of Shielded Resolvers'}).to_latex(index=False,float_format="%.0f"))

#######################
######  TABLE 4  ######
#######################
\begin{tabular}{rrr}
\toprule
ASN & \# of Transp. Fwds. & \# of Shielded Resolvers \\
\midrule
4812 & 17923 & 9880 \\
5769 & 8377 & 4256 \\
209 & 5576 & 2446 \\
19901 & 2050 & 744 \\
5483 & 1724 & 578 \\
\bottomrule
\end{tabular}



In [12]:
fnames = glob.glob(f"{data_dir}/any_dnssec_support/udp_A_DNSSEC_verisign.com_*.csv.gz")

dnssec_df = pl.concat(
    [pl.read_csv(file, separator=";", has_header=False, new_columns=["id","ip_request","ip_response","a_record","ts_request","port","dnsid","dns_pkt_size","dns_recs","dns_flags"]).with_columns(
        pl.lit(file.split("_")[-1].split(".")[0]).alias("resolver_type")
    ) for idx, file in enumerate(fnames)]
)
 
dnssec_df = dnssec_df.with_columns(
    pl.when(pl.col("dns_recs").str.contains("RRSIG"))
    .then(pl.lit(True))
    .otherwise(pl.lit(False))
    .alias("has_dnssec")
)

dnssec_df = dnssec_df.group_by("ip_response","has_dnssec","resolver_type").agg(pl.len())
dnssec_df_filtered_has = dnssec_df.filter(pl.col("has_dnssec"))
dnssec_df_filtered_has_not = dnssec_df.filter(~pl.col("ip_response").is_in(dnssec_df_filtered_has["ip_response"]))
dnssec_df = pl.concat([dnssec_df_filtered_has,dnssec_df_filtered_has_not])
dnssec_df = dnssec_df.group_by("has_dnssec","resolver_type").agg(
    pl.len().alias("dnssec_amount")
)


In [13]:
fnames = glob.glob(f"{data_dir}/any_dnssec_support/udp_ANY_EDNS0_verisign.com_*.csv.gz")

any_df = pl.concat(
    [pl.read_csv(file, separator=";", has_header=False, new_columns=["id","ip_request","ip_response","a_record","ts_request","port","dnsid","dns_pkt_size","dns_recs","dns_flags"]).with_columns(
        pl.lit(file.split("_")[-1].split(".")[0]).alias("resolver_type")
    ) for idx, file in enumerate(fnames)]
)
any_df = any_df.with_columns(
    pl.when(pl.col("dns_pkt_size") > 100)
    .then(pl.lit(True))
    .otherwise(pl.lit(False))
    .alias("has_any")
)

any_df = any_df.group_by("ip_response","has_any","resolver_type").agg(pl.len())

any_df_filtered_has = any_df.filter(pl.col("has_any"))
any_df_filtered_has_not = any_df.filter(~pl.col("ip_response").is_in(dnssec_df_filtered_has["ip_response"]))
any_df = pl.concat([any_df_filtered_has,any_df_filtered_has_not])
any_df = any_df.group_by("has_any","resolver_type").agg(
    pl.len().alias("any_amount")
)

In [14]:
# all known response addresses are already directly accessible
response_addresses2 = (
    scan_df_aug
    .filter(pl.col("response_type") != "Transparent Forwarder")
    .select("ip_response")
    .unique()
)
# we might know some resolver addresses through the a-record as well, but they are part of the shielded resolvers since these resolvers are not publically accessible

# mapping of transparent forwarders to shielded resolvers
tfwd_shielded_df2 = (
    scan_df_aug
    .filter(pl.col("response_type") == "Transparent Forwarder")
    .filter(~pl.col("ip_response").is_in(response_addresses2))
)
# ip addresses of only the shielded resolvers themselves
shielded_resolvers2 = tfwd_shielded_df2.select("ip_response").unique()

In [15]:
total_shielded = shielded_resolvers2.n_unique()
shielded_any = any_df.filter((pl.col('has_any')) & (pl.col('resolver_type')=='shielded')).select(pl.col('any_amount')).item()
shielded_noany = any_df.filter((~pl.col('has_any')) & (pl.col('resolver_type')=='shielded')).select(pl.col('any_amount')).item()

shielded_dnssec = dnssec_df.filter((pl.col('has_dnssec')) & (pl.col('resolver_type')=='shielded')).select(pl.col('dnssec_amount')).item()
shielded_nodnssec = dnssec_df.filter((~pl.col('has_dnssec')) & (pl.col('resolver_type')=='shielded')).select(pl.col('dnssec_amount')).item()

total_unshielded = scan_df_aug.filter(pl.col('response_type')=='Resolver').n_unique()
unshielded_any = any_df.filter((pl.col('has_any')) & (pl.col('resolver_type')=='unshielded')).select(pl.col('any_amount')).item()
unshielded_noany = any_df.filter((~pl.col('has_any')) & (pl.col('resolver_type')=='unshielded')).select(pl.col('any_amount')).item()

unshielded_dnssec = dnssec_df.filter((pl.col('has_dnssec')) & (pl.col('resolver_type')=='unshielded')).select(pl.col('dnssec_amount')).item()
unshielded_nodnssec = dnssec_df.filter((~pl.col('has_dnssec')) & (pl.col('resolver_type')=='unshielded')).select(pl.col('dnssec_amount')).item()

table_tmp = pd.DataFrame({'Query':['DNSSEC']*3+['ANY']*3,
                          'Support':[True,False,'n/a']*2,
                          'Shielded Res. [#]':[shielded_dnssec,
                                               shielded_nodnssec,
                                               total_shielded-shielded_dnssec-shielded_nodnssec,
                                               shielded_any,
                                               shielded_noany,
                                               total_shielded-shielded_any-shielded_noany
                                              ],
                          'Shielded Res. [%]':[round(shielded_dnssec/total_shielded*100,1),
                                               round(shielded_nodnssec/total_shielded*100,1),
                                               round((total_shielded-shielded_dnssec-shielded_nodnssec)/total_shielded*100,1),
                                               round(shielded_any/total_shielded*100,1),
                                               round(shielded_noany/total_shielded*100,1),
                                               round((total_shielded-shielded_any-shielded_noany)/total_shielded*100,1)
                                              ],
                          'Open Res. [#]':[unshielded_dnssec,
                                               unshielded_nodnssec,
                                               total_unshielded-unshielded_dnssec-unshielded_nodnssec,
                                               unshielded_any,
                                               unshielded_noany,
                                               total_unshielded-unshielded_any-unshielded_noany
                                              ],
                          'Open Res. [%]':[round(unshielded_dnssec/total_unshielded*100,1),
                                               round(unshielded_nodnssec/total_unshielded*100,1),
                                               round((total_unshielded-unshielded_dnssec-unshielded_nodnssec)/total_unshielded*100,1),
                                               round(unshielded_any/total_unshielded*100,1),
                                               round(unshielded_noany/total_unshielded*100,1),
                                               round((total_unshielded-unshielded_any-unshielded_noany)/total_unshielded*100,1)
                                              ]
                         })

In [16]:
print("#######################")
print("######  TABLE 5  ######")
print("#######################")
print(table_tmp.to_latex(index=False))

#######################
######  TABLE 5  ######
#######################
\begin{tabular}{llrrrr}
\toprule
Query & Support & Shielded Res. [#] & Shielded Res. [%] & Open Res. [#] & Open Res. [%] \\
\midrule
DNSSEC & True & 5989 & 23.600000 & 22092 & 65.000000 \\
DNSSEC & False & 15197 & 59.900000 & 4630 & 13.600000 \\
DNSSEC & n/a & 4168 & 16.400000 & 7246 & 21.300000 \\
ANY & True & 3791 & 15.000000 & 24146 & 71.100000 \\
ANY & False & 505 & 2.000000 & 884 & 2.600000 \\
ANY & n/a & 21058 & 83.100000 & 8938 & 26.300000 \\
\bottomrule
\end{tabular}



In [17]:
anycast_ip_addresses = [
    "8.8.8.8",
    "8.8.4.4",
    "1.1.1.1",
    "1.0.0.1",
    "208.67.222.222",
    "9.9.9.9",
    "45.90.28.118",
    "76.76.2.0",
    "156.154.70.1",
    "199.85.126.10",
    "64.6.64.6",
    "4.2.2.1",
    "74.82.42.42",
    "185.228.168.9",
    "195.46.39.39",
    "8.26.56.26",
    "94.140.14.14",
    "45.11.45.11",
    "216.146.35.35",
    "223.5.5.5",
    "119.29.29.29",
    "101.226.4.6",
    "180.76.76.76",
    "114.114.114.114",
    "1.2.4.8",
    "205.171.2.26",
    "149.112.121.10",
    "80.80.80.80",
    "77.88.8.8",
    "193.58.251.251"
]
anycast_ip_addresses = ['1.1.1.1', '8.8.8.8', '208.67.222.222', '77.88.8.8',
       '114.114.114.114', '223.5.5.5', '4.2.2.1', '195.46.39.39',
       '119.29.29.29', '1.2.4.8', '193.58.251.251']

In [18]:
anycast_df = scan_df.filter((pl.col('ip_response')!=pl.col('a_record')) & (pl.col('ip_response').is_in(anycast_ip_addresses)) & (pl.col('response_type')=='Transparent Forwarder')).to_pandas()#.org_response.unique()

In [19]:
anycast_df['ip_request24'] = anycast_df.ip_request.apply(lambda ip: '.'.join(ip.split('.')[:-1]))
anycast_df['arecord24'] = anycast_df.a_record.apply(lambda ip: '.'.join(ip.split('.')[:-1]))

In [20]:
anycast_df = pl.from_pandas(anycast_df)

In [21]:
print("#######################")
print("######  TABLE 6  ######")
print("#######################")
print(anycast_df.group_by('ip_response').agg(
    pl.col('country_request').n_unique().alias('Countries [#]'),
    pl.col('ip_request').n_unique().alias('Tfwd [#]'),
    pl.col('ip_request24').n_unique().alias('Tfwd /24 [#]'),
    pl.col('a_record').n_unique().alias('Anycast Infrastructure [#]'),
    pl.col('arecord24').n_unique().alias('Anycast Infrastructure /24 [#]'),
).sort(['Countries [#]','Tfwd [#]'],descending=True).to_pandas().to_latex(index=False))

#######################
######  TABLE 6  ######
#######################
\begin{tabular}{lrrrrr}
\toprule
ip_response & Countries [#] & Tfwd [#] & Tfwd /24 [#] & Anycast Infrastructure [#] & Anycast Infrastructure /24 [#] \\
\midrule
8.8.8.8 & 74 & 341447 & 5174 & 1889 & 139 \\
1.1.1.1 & 46 & 48313 & 953 & 216 & 151 \\
208.67.222.222 & 26 & 14464 & 227 & 224 & 42 \\
195.46.39.39 & 2 & 19 & 4 & 2 & 2 \\
4.2.2.1 & 2 & 9 & 2 & 9 & 3 \\
77.88.8.8 & 1 & 725 & 17 & 119 & 5 \\
114.114.114.114 & 1 & 70 & 26 & 5 & 3 \\
223.5.5.5 & 1 & 62 & 22 & 16 & 16 \\
119.29.29.29 & 1 & 3 & 3 & 3 & 3 \\
193.58.251.251 & 1 & 3 & 1 & 1 & 1 \\
1.2.4.8 & 1 & 2 & 1 & 2 & 1 \\
\bottomrule
\end{tabular}



In [22]:
rate_pub_resolver_multi_pop_files

'./data/ratelimits-multiple-pops'

In [23]:
root_path = rate_pub_resolver_multi_pop_files
rl_resolver_multipop = pd.DataFrame([])
for cc_folder in os.listdir(root_path):
    scan_folders = sorted(os.listdir(os.path.join(root_path, cc_folder)))
    for idx, scan_folder in enumerate(scan_folders):
        scan_path = os.path.join(root_path, cc_folder, scan_folder)
        if not os.path.isdir(scan_path) or 'ipynb_checkpoints' in scan_path:
            continue
        tmp = pd.read_csv(os.path.join(scan_path,'rates.csv'), sep=';', names=['resolver_ip','max_rate', 'end_rate'])
        ts = scan_folder.split('_')[0]
        tmp['date'] = pd.to_datetime(ts)
        tmp['country'] = cc_folder
        rl_resolver_multipop = pd.concat([rl_resolver_multipop,tmp],ignore_index=True)

In [24]:
formatted_df = rl_resolver_multipop.groupby(['resolver_ip','country'])['max_rate'].mean().unstack(level=-1).merge(pd.read_csv(f'{data_dir}/pub_dns_servers/public_dns_servers.txt',sep=';'), left_on='resolver_ip', right_on='resolver_addr').set_index(['resolver_name', 'resolver_addr']).sort_index()

max_rate_per_resolver = formatted_df.max(axis=1).groupby(level='resolver_name').max()
sorted_resolver_order = max_rate_per_resolver.sort_values(ascending=False).index
formatted_df = formatted_df.reindex(sorted_resolver_order, level='resolver_name')

formatted_df = formatted_df[(formatted_df != 0).any(axis=1)]
formatted_df.index.names = ['Resolver name', 'IPv4-address']
formatted_df.columns = pd.MultiIndex.from_product([['Max. rate on avg.'], formatted_df.columns])
formatted_df = formatted_df.astype(int)
formatted_df = formatted_df[formatted_df.index.get_level_values(0).isin(['Google','Cloudflare','OpenDNS','Verisign','AliDNS','CNNIC SDNS','114DNS','DNSPod Public DNS+','Yandex.DNS'])]
order_mapping = {'Google':1,'Cloudflare':2,'OpenDNS':3,'Verisign':4,'AliDNS':5,'CNNIC SDNS':6,'114DNS':7,'DNSPod Public DNS+':8,'Yandex.DNS':9}
formatted_df = formatted_df.sort_index(level=0,key=lambda name: name.map(order_mapping))
tex_table = formatted_df.to_latex(multirow=True, index=True)
print("#######################")
print("######  TABLE 7  ######")
print("#######################")
print(tex_table)

#######################
######  TABLE 7  ######
#######################
\begin{tabular}{llrrr}
\toprule
 &  & \multicolumn{3}{r}{Max. rate on avg.} \\
 &  & BRA & GER & IND \\
Resolver name & IPv4-address &  &  &  \\
\midrule
\multirow[t]{2}{*}{Google} & 8.8.4.4 & 2177 & 1773 & 1597 \\
 & 8.8.8.8 & 2170 & 1744 & 1608 \\
\cline{1-5}
\multirow[t]{2}{*}{Cloudflare} & 1.0.0.1 & 1008 & 874 & 754 \\
 & 1.1.1.1 & 863 & 822 & 796 \\
\cline{1-5}
\multirow[t]{2}{*}{OpenDNS} & 208.67.220.220 & 832 & 829 & 769 \\
 & 208.67.222.222 & 785 & 733 & 773 \\
\cline{1-5}
\multirow[t]{2}{*}{Verisign} & 64.6.64.6 & 192 & 199 & 192 \\
 & 64.6.65.6 & 2254 & 1796 & 1401 \\
\cline{1-5}
\multirow[t]{2}{*}{AliDNS} & 223.5.5.5 & 1894 & 1753 & 1341 \\
 & 223.6.6.6 & 1882 & 1729 & 1506 \\
\cline{1-5}
\multirow[t]{2}{*}{CNNIC SDNS} & 1.2.4.8 & 1932 & 426 & 740 \\
 & 210.2.4.8 & 1194 & 1358 & 223 \\
\cline{1-5}
\multirow[t]{2}{*}{114DNS} & 114.114.114.114 & 58 & 55 & 54 \\
 & 114.114.115.115 & 58 & 54 & 51 \\
\cline{1

In [25]:
orchestration_df = pl.read_csv(f'{data_dir}/tfwds_to_public_dns_servers/2024-10-22_11-27-12_rm-direct_dm-constant_incr-2000ms_max-rate-3000pps/rates.csv',separator=';',has_header=False,new_columns=['Anycast IP address','Packet rate [pps]','Single count'])\
.select(['Anycast IP address','Packet rate [pps]']).sort(by='Packet rate [pps]',descending=True).filter(pl.col('Anycast IP address')!='9.9.9.9')

In [26]:
anycast_company = {"8.8.8.8":'Google',"8.8.4.4":'Google',"1.1.1.1":"Cloudflare","1.0.0.1":"Cloudflare","208.67.222.222":"OpenDNS","208.67.220.220":"OpenDNS"}

In [27]:
custom_order={'Google':1,'Cloudflare':2,'OpenDNS':3}

In [28]:
orchestration_df = orchestration_df.with_columns(
    pl.col('Anycast IP address').replace(anycast_company).alias('Company')
)[['Company','Anycast IP address','Packet rate [pps]']]

In [29]:
orchestration_df = orchestration_df.insert_column(3,pl.Series("Transp. fwd. [#]", [10,7,8,5,7,5]))

In [30]:
orchestration_df = pl.from_pandas(orchestration_df.to_pandas().sort_values(by='Company',key=lambda x:x.map(custom_order))).insert_column(4,pl.Series('Resolver Packet rate [pps]',[1744, 1773, 822, 874, 733, 829])).to_pandas()

In [32]:
print("#######################")
print("######  TABLE 8  ######")
print("#######################")
print(orchestration_df.to_latex(index=False))

#######################
######  TABLE 8  ######
#######################
\begin{tabular}{llrrr}
\toprule
Company & Anycast IP address & Packet rate [pps] & Transp. fwd. [#] & Resolver Packet rate [pps] \\
\midrule
Google & 8.8.8.8 & 24815 & 10 & 1744 \\
Google & 8.8.4.4 & 17153 & 7 & 1773 \\
Cloudflare & 1.1.1.1 & 6002 & 8 & 822 \\
Cloudflare & 1.0.0.1 & 3600 & 5 & 874 \\
OpenDNS & 208.67.222.222 & 5660 & 5 & 733 \\
OpenDNS & 208.67.220.220 & 3950 & 7 & 829 \\
\bottomrule
\end{tabular}

